# Subset, Clean and Concat funky lat longs

In [1]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import to_snakecase
from dla_utils import _dla_utils
from siuba import *
from shared_utils import geography_utils

import altair as alt

import _data_cleaning
import _report_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_4235/2916912562.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling 

In [2]:
pd.set_option("display.max_columns",500)

In [3]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [4]:
df_all = _report_utils.read_in_joined_data()

In [5]:
cycle = 5

In [6]:
df = df_all>>filter(_.project_cycle==cycle)

In [7]:
len(df)

450

In [10]:
df_map= (df >>select(_.awarded, _.project_app_id, _.project_cycle, _.data_origin, _.geometry,
               _.a1_imp_agcy_city, _.a1_imp_agcy_name, _.a1_proj_partner_agcy, 
               _.assembly_district, _.congressional_district, _.senate_district,
              _.a2_county, _.a2_info_proj_descr, _.a2_info_proj_loc, _.a2_info_proj_name,
               _.a2_mop_uza_population, _.a2_mpo, _.a1_imp_agcy_street, _.a3_proj_type, 
               _.a3_proj_type, _['total_atp_$'], _.a2_proj_lat, _.a2_proj_long))

In [11]:
df_map = (geography_utils.create_point_geometry(df_map, longitude_col = 'a2_proj_long', latitude_col = 'a2_proj_lat'))

In [12]:
_report_utils.join_state_with_points(df_map)

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long,index_right,State,point_check
180,N,03-El Dorado County-1,5,Application,POINT (-120.82612 38.70348),Placerville,El Dorado County,NaN,5,4,1,El Dorado,Construct a Class I multi-use grade-separated ...,On Missouri Flat Rd. between Golden Center Dr....,El Dorado Trail / Missouri Flat Road Pedestria...,Project is located within one of the ten large...,SACOG,2850 Fairlane Court,Infrastructure - Medium,NaN,38.70,-120.83,0.00,CA,Point In State
99,N,"6-Fresno, City of-2",5,Application,POINT (-119.80835 36.75038),Fresno,"Fresno, City of",NaN,"23, 31",16,Needs Manual Assistance,Fresno,Palm and Belmont: 3.5 miles of Class IV with C...,Bikeway on Palm Avenue: just north of Dakota A...,Palm and Belmont Protected Bikeway Project,Project is located within one of the ten large...,COFCG,2600 Fresno Street,Infrastructure - Small,NaN,36.75,-119.81,0.00,CA,Point In State
211,N,"6-Fresno, City of-3",5,Application,POINT (-119.77249 36.76316),Fresno,"Fresno, City of",NaN,31,16,Needs Manual Assistance,Fresno,"Install signal/scramble at Chestnut & Weldon, ...",The intersections of Chestnut and Weldon and F...,"Cross, Walk & Roll! SRTS in Central Fresno",Project is located within one of the ten large...,COFCG,2600 Fresno Street,Infrastructure + NI - Small,NaN,36.76,-119.77,0.00,CA,Point In State
275,N,6-Kern Council of Governments-1,5,Application,POINT (-119.01526 35.37227),Bakersfield,Kern Council of Governments,NaN,"32, 34, 36","21, 23","14, 16",Kern,Identified disadvantaged communities in Kern C...,"DACs in Arvin, Bakersfield, California City, D...",Safe Routes for Cyclists in Kern County's Disa...,Project is located within one of the ten large...,KCOG,"1401 19th Street, Suite 300",Non-Infrastructure,NaN,35.37,-119.02,0.00,CA,Point In State
415,N,6-Kern County - D6-1,5,Application,POINT (-118.89088 35.43151),Bakersfield,Kern County - D6,NaN,34,23,16,Kern,Rehabilitate & resurface the Kern River Parkwa...,Kern River Parkway Multi-Use Path between Mano...,Kern River Parkway Multi-Use Path Safety Impro...,Project is located within one of the ten large...,KCOG,"2700 M Street, Suite 400",Infrastructure - Small,NaN,35.43,-118.89,0.00,CA,Point In State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,Y,"7-Los Angeles, City of-5",5,Funded,POINT (-118.45719 34.22689),Los Angeles,"Los Angeles, City of",NaN,46,29,18,Los Angeles,Transform one of the City’s most traffic-stres...,The one-quarter mile radius around the Panoram...,SRTS Panorama City Elementary School Project,Project is located within one of the ten large...,SCAG,"100 S. Main Street, 9th Floor",NaN,"6,149,000.00",34.23,-118.46,0.00,CA,Point In State
41,Y,"6-Huron, City of-1",5,Funded,POINT (-120.10304 36.21027),Huron,"Huron, City of",NaN,31,21,12,Fresno,"The project will add Class II, III and IV bike...",The proposed project will take place on Lassen...,City of Huron Bicyclist and Pedestrian Safety ...,Project is located within one of the ten large...,COFCG,PO Box 339,NaN,"1,769,000.00",36.21,-120.10,0.00,CA,Point In State
38,Y,10-Tuolumne County-1,5,Funded,POINT (120.23264 37.58579),Sonora,Tuolumne County,NaN,8,4,8,Tuolumne,"Project includes approximately 4,200 linear fe...","On 5th Ave, from Jamestown Rd, cross 108 to 7t...",Jamestown Community Connectivity Project,Project is located outside one of the ten larg...,Caltrans,2 S. Green Street,NaN,"2,071,000.00",37.59,120.23,NaN,None,Point Not In State
40,Y,"7-Long Beach, City of-1",5,Funded,POINT (-118.19233 33.77071),Long Beach,"Long Beach, City of",NaN,70,47,33,Los Angeles,Transform Downtown Long Beach into a pedestria...,Select intersections in Downtown Long Beach 

In [13]:
(_report_utils.join_state_with_points(df_map)).point_check.value_counts()

Point In State        320
Point Not In State    130
Name: point_check, dtype: int64

In [14]:
## subset the rows that have incorrect lat longs
need_help = ((_report_utils.join_state_with_points(df_map))>>filter(_.point_check=='Point Not In State'))

In [15]:
#check to make sure we have all rows
need_help.point_check.value_counts()

Point Not In State    130
Name: point_check, dtype: int64

In [16]:
##subset those to just have points in state. 
##will add the fixed geometries back to this in after
joined = ((_report_utils.join_state_with_points(df_map))>>filter(_.point_check=='Point In State'))

In [17]:
#check to make sure we have all rows
joined.point_check.value_counts()

Point In State    320
Name: point_check, dtype: int64

In [18]:
need_help>>arrange(-_.a2_proj_lat)

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long,index_right,State,point_check
283,N,"7-Pasadena, City of-1",5,Application,POINT (1883071.21000 6516102.76300),Pasadena,"Pasadena, City of",NaN,41,27,25,Los Angeles,The goal of the project is to develop ATP for ...,"Northwest Pasadena: 210 Freeway to the South, ...",Northwest Pasadena Active Transportation Plan ...,Project is located within one of the ten large...,SCAG,221 East Walnut Suite 210,Plan,NaN,"6,516,102.76","1,883,071.21",NaN,None,Point Not In State
260,N,"7-Pasadena, City of-2",5,Application,POINT (1883071.21000 6516102.76300),Pasadena,"Pasadena, City of",NaN,41,27,25,Los Angeles,Replacement of existing standard crosswalks wi...,"Northwest Pasadena: 210 Freeway to the south, ...",Northwest Pasadena Continental Crosswalk Imple...,Project is located within one of the ten large...,SCAG,221 East Walnut Street,Infrastructure - Small,NaN,"6,516,102.76","1,883,071.21",NaN,None,Point Not In State
13,Y,11-San Diego Association of Governments (SANDA...,5,Funded,POINT (1170624.00000 324511.00000),San Diego,San Diego Association of Governments (SANDAG),City of San Diego,"78, 80","51, 53",40,San Diego,Buffered bike lanes; traffic calming and ped a...,The Orange Family Friendly Street Project is 2...,Orange Family Friendly Street Project,Project is located within one of the ten large...,SANDAG,"401 B Street, Suite 800",NaN,"4,317,000.00","324,511.00","1,170,624.00",NaN,None,Point Not In State
382,N,"7-Baldwin Park, City of-1",5,Application,POINT (117.96491 43.10680),Baldwin Park,"Baldwin Park, City of",NaN,48,32,22,Los Angeles,"Install Traffic Control Devices, HAWK System S...",Baldwin Park Boulevard from City limit (250 ft...,Baldwin Park Blvd Bike Lane Project,Project is located within one of the ten large...,SCAG,14403 E. Pacific Ave,Infrastructure - Medium,NaN,43.11,117.96,NaN,None,Point Not In State
317,N,2-Modoc County-1,5,Application,POINT (120.17330 41.52910),Alturas,Modoc County,NaN,1,1,1,Modoc,"Construct/replace sidewalks, implement high vi...","In the township of Cedarville, On Bonner Stree...",Surprise Valley School Safety and Community Co...,Project is located outside one of the ten larg...,MCTC,202 West 4th Street,Infrastructure - Medium,NaN,41.53,120.17,NaN,None,Point Not In State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Y,"11-National City, City of-3",5,Funded,POINT (32.65651 32.65651),National City,"National City, City of",NaN,80,51,40,San Diego,"Construct Class IV and II bicycle facilities, ...","National City, CA. Highland Avenue from 30th ...",Highland Avenue Inter-City Bike Connection,Project is located within one of the ten large...,SANDAG,1243 National City Boulevard,NaN,"1,343,000.00",32.66,32.66,NaN,None,Point Not In State
277,N,"11-Chula Vista, City of-1",5,Application,POINT (117.09559 32.63669),Chula Vista,"Chula Vista, City of",NaN,80,51,40,San Diego,Construct 12-foot wide multi-use path for pede...,East of San Diego Bay in northwestern Chula Vi...,"F Street Promenade Phase I, from Bay Boulevard...",Project is located within one of the ten large...,SANDAG,Engineering Department - 276 Fourth Avenue,Infrastructure - Large,NaN,32.64,117.10,NaN,None,Point Not In State
43,Y,"11-Imperial Beach, City of-1",5,Funded,POINT (117.11441 32.57646),Imperial Beach,"Imperial Beach, City of",NaN,78,51,40,San Diego,1.3 miles of bicycle and pedestrian improvemen...,9th Street in Imperial Beach between the south...,9th St Active Transportation Corridor,Project is located within one of the ten large...,SANDAG,495 10th Street,NaN,"3,018,000.00",32.58,117.11,NaN,None,Point Not In State
155,N,"11-Encinitas, City of-1",5,Application,PO

In [19]:
## get those that have a negative long
need_help_pt1 = need_help>>filter(_.a2_proj_long<0)

In [20]:
## get those with a positive long (to change into a negative long)
need_help_pt2 = need_help>>filter(_.a2_proj_long>0)

In [21]:
## get those with abnormal lats 
#need_help_pt3 = need_help_pt2>>filter(_.a2_proj_lat>300)
need_help_pt3 = need_help_pt2[(need_help_pt2['a2_proj_lat'] > 300) | (need_help_pt2['a2_proj_long'] < 50)]

In [22]:
## get those with normal lats and positive longs
#need_help_pt2 = need_help_pt2>>filter(_.a2_proj_lat<300)
need_help_pt2 = need_help_pt2[(need_help_pt2['a2_proj_lat'] < 300) & (need_help_pt2['a2_proj_long'] > 50)]

In [23]:
## concat need_help1 and need-help3 to have a some that we can use the implementing agency's city. 
need_help_pt1 = (pd.concat([need_help_pt1, need_help_pt3])).drop_duplicates()

### Change Long to negative for need_help_pt_2

In [24]:
need_help_pt2['a2_proj_long'] = need_help_pt2['a2_proj_long'] * (-1)

In [25]:
need_help_pt2 = (geography_utils.create_point_geometry(need_help_pt2, longitude_col = 'a2_proj_long', latitude_col = 'a2_proj_lat'))

In [26]:
(_report_utils.join_state_with_points(need_help_pt2)).point_check.value_counts()

Point In State        117
Point Not In State      5
Name: point_check, dtype: int64

In [27]:
need_help_pt2 = (_report_utils.join_state_with_points(need_help_pt2))

In [28]:
## add in those with points not in state to the other entries
need_help_pt1 = pd.concat([need_help_pt1, (need_help_pt2>>filter(_.point_check=='Point Not In State'))])

In [29]:
## get those just with in state points as finished.
need_help_pt2_done = need_help_pt2>>filter(_.point_check=='Point In State')

### Using Place Names City data to get correct lat long

In [30]:
need_help_pt1 = _report_utils.get_latlong_from_placenames(need_help_pt1, 'a1_imp_agcy_city')

In [31]:
need_help_pt1_done = (_report_utils.join_state_with_points(need_help_pt1))

In [32]:
need_help_pt1_done.point_check.value_counts()

Point In State    13
Name: point_check, dtype: int64

## Concat fixed lat long together

In [33]:
#concat together
fixed = pd.concat([need_help_pt1_done,need_help_pt2_done ])

In [34]:
fixed

,awarded,project_app_id,project_cycle,data_origin,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long,geometry,index_right,State,point_check
0,N,"7-Cerritos, City of-2",5,Application,Cerritos,"Cerritos, City of",City of Lakewood,58,38,32,Los Angeles,Install missing Class II bike lanes and sidewa...,Del Amo Boulevard from I-605 to Denni Street i...,Del Amo Boulevard Bridge Replacement and Signa...,Project is located within one of the ten large...,SCAG,18125 Bloomfield Ave,Infrastructure - Large,NaN,+33.8677431,-118.0694719,POINT (-118.06947 33.86774),0.00,CA,Point In State
1,N,5-San Luis Obispo County-2,5,Application,San Luis Obispo,San Luis Obispo County,NaN,35,24,17,San Luis Obispo,"Construct a 1.0-mile, 12-foot wide, Class I, m...",Situated along unsanctioned coastal access poi...,Morro Bay to Cayucos Multi-Use Trail,Project is located outside one of the ten larg...,SLOCOG,1055 Monterey Street,Infrastructure - Large,NaN,+35.2639540,-120.6611260,POINT (-120.66113 35.26395),0.00,CA,Point In State
2,N,9-California Department of Transportation-6,5,Application,Bishop,California Department of Transportation,NaN,26,8,8,Inyo,Construction of ADA accessible sidewalk with o...,This project is located in Inyo County outside...,SR 168 (W. Line Street) Sidewalk Extension - B...,Project is located outside one of the ten larg...,Caltrans,500 S. Main Street,Infrastructure - Medium,NaN,+37.3663813,-118.3958082,POINT (-118.39581 37.36638),0.00,CA,Point In State
3,N,1-Round Valley Indians Tribe-1,5,Application,Covelo,Round Valley Indians Tribe,NaN,2,2,2,Mendocino,The project entails constructing 4775 linear f...,"The project is in Round Valley, a remote locat...",Foothill Blvd. Safe Routes to School Sidewalk ...,Project is located outside one of the ten larg...,Caltrans,77826 Covelo Road,Infrastructure - Small,NaN,+39.8001872,-123.2526012,POINT (-123.25260 39.80019),0.00,CA,Point In State
4,N,"7-Pasadena, City of-1",5,Application,Pasadena,"Pasadena, City of",NaN,41,27,25,Los Angeles,The goal of the project is to develop ATP for ...,"Northwest Pasadena: 210 Freeway to the South, ...",Northwest Pasadena Active Transportation Plan ...,Project is located within one of the ten large...,SCAG,221 East Walnut Suite 210,Plan,NaN,+34.1605920,-118.1395930,POINT (-118.13959 34.16059),0.00,CA,Point In State
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,Y,"6-Delano, City of-1",5,Funded,Delano,"Delano, City of",NaN,32,21,14,Kern,"Construct 68 curb ramps, 87 crosswalks, advanc...","Project is located in Delano, a severely disad...",ATP-5 SRTS Intersection Enhancement and NI Wor...,Project is located within one of the ten large...,KCOG,1015 Eleventh Avenue,NaN,"1,164,000.00",35.77,-119.25,POINT (-119.24710 35.76880),0.00,CA,Point In State
29,Y,4-Contra Costa County-2,5,Funded,Martinez,Contra Costa County,NaN,14,11,7,Contra Costa,"Construct two-way cycle track, ADA-compliant c...","Bailey Road in unincorporated Bay Point, bound...",North Bailey Road Active Transportation Corridor,Project is located within one of the ten large...,MTC,255 Glacier Drive,NaN,"6,159,000.00",38.02,-121.94,POINT (-121.94196 38.02416),0.00,CA,Point In State
28,Y,10-Mariposa County-2,5,Funded,Mariposa,Mariposa County,NaN,5,4,8,Mariposa,The Mariposa Creek Parkway provides an off-str...,The project is located along Mariposa Creek in...,Mariposa Creek Parkway,Project is located outside one of the ten larg...,MCTC,4639 Ben Hur Road,NaN,"4,415,000.00",37.49,-119.97,POINT (-119.97080 37.48700),0.00,CA,Point In State
1,Y,"7-South El Monte, City of-1",5,Funded,South El Monte,"South El Monte, City of",NaN,57,38,22,Los Angeles,This project focuses on school and pedestrian ...,The project is fully in the City of South El 

In [35]:
##check map
#fixed.explore("data_origin", cmap="tab20b")  

## Put it all into a Function

* move to _report_utils

In [36]:
# def fix_geom_issues(df, subset_col_list):
    
#     ## susbet dataframe based on a list of columns
#     df_map = df[df.columns.intersection(subset_col_list)]
#     ## create geometry points from lat long col
#     df_map = (geography_utils.create_point_geometry(df_map, longitude_col = 'a2_proj_long', latitude_col = 'a2_proj_lat'))
    
#     ##use join_state_with_points to determine which points are in state or not 
#     ##will add the fixed geometries back to this in after
#     joined = ((_report_utils.join_state_with_points(df_map))>>filter(_.point_check=='Point In State'))
#     ## subset the rows that have incorrect lat longs 
#     need_help = ((_report_utils.join_state_with_points(df_map))>>filter(_.point_check=='Point Not In State'))
    
#     ## get those that have a negative long
#     need_help_pt1 = need_help>>filter(_.a2_proj_long<0)
#     ## get those with a positive long (to change into a negative long)
#     need_help_pt2 = need_help>>filter(_.a2_proj_long>0)
    
#     ## get those with abnormal lats 
#     need_help_pt3 = need_help_pt2[(need_help_pt2['a2_proj_lat'] > 300) | (need_help_pt2['a2_proj_long'] < 50)]
#     ## get those with normal lats and positive longs
#     need_help_pt2 = need_help_pt2[(need_help_pt2['a2_proj_lat'] < 300) & (need_help_pt2['a2_proj_long'] > 50)]
    
#     ## concat need_help1 and need-help3 to have a some that we can use the implementing agency's city. 
#     need_help_pt1 = (pd.concat([need_help_pt1, need_help_pt3])).drop_duplicates()
    
#     ## multiply longs by -1 to get correct long 
#     need_help_pt2['a2_proj_long'] = need_help_pt2['a2_proj_long'] * (-1)
#     ## redo geom points for fixed longs
#     need_help_pt2 = (geography_utils.create_point_geometry(need_help_pt2, longitude_col = 'a2_proj_long', latitude_col = 'a2_proj_lat'))
    
#     ## recheck if points are fixed to be in state
#     need_help_pt2 = (_report_utils.join_state_with_points(need_help_pt2))
    
#     ## add in those with points not in state to the other entries
#     need_help_pt1 = pd.concat([need_help_pt1, (need_help_pt2>>filter(_.point_check=='Point Not In State'))])
    
#     ## get those just with in state points as finished.
#     need_help_pt2_done = need_help_pt2>>filter(_.point_check=='Point In State')
    
#     ## use place names
#     need_help_pt1 = _report_utils.get_latlong_from_placenames(need_help_pt1, 'a1_imp_agcy_city')
#     ## make sure points are in state again
#     need_help_pt1_done = (_report_utils.join_state_with_points(need_help_pt1))
    
#     ##concat together
#     fixed = pd.concat([need_help_pt1_done, need_help_pt2_done])
    
#     ## assert there are no "Point Not In State" Values in columns
#     assert (len(fixed>>filter(_.point_check=='Point Not In State')))==0
    
#     final_df = pd.concat([joined, fixed])

#     return final_df

In [37]:
mapsubset_cols= ['awarded','project_app_id', 'project_cycle', 'data_origin', 'geometry',
               'a1_imp_agcy_city','a1_imp_agcy_name','a1_proj_partner_agcy', 
               'assembly_district','congressional_district','senate_district',
              'a2_county','a2_info_proj_name','a3_proj_type', 'total_atp_$', 'a2_proj_lat','a2_proj_long']

In [38]:
##checking with report utils
df2 = _report_utils.fix_geom_issues(df, mapsubset_cols)

In [39]:
df2.point_check.value_counts()

Point In State    450
Name: point_check, dtype: int64

In [40]:
##check map
#df2.explore("data_origin", cmap="tab20b")